In [2]:
import pandas as pd

elec_2022 = pd.read_csv("../dataset/Elections Dataset/pres_precinct_data_2022.csv")
elec_2016 = pd.read_csv("../dataset/Elections Dataset/vp_precinct_data_2016.csv")
elec_2019 = pd.read_json("../dataset/Elections Dataset/2019/philippine_provinces_cities_municipalities_and_barangays_2019v2.json")


In [3]:
elec_2019.dtypes

01       object
02       object
03       object
4A       object
4B       object
05       object
06       object
07       object
08       object
09       object
10       object
11       object
12       object
13       object
BARMM    object
CAR      object
NCR      object
dtype: object

In [4]:
elec_2019

,01,02,03,4A,4B,05,06,07,08,09,10,11,12,13,BARMM,CAR,NCR
region_name,REGION I,REGION II,REGION III,REGION IV-A,REGION IV-B,REGION V,REGION VI,REGION VII,REGION VIII,REGION IX,REGION X,REGION XI,REGION XII,REGION XIII,BARMM,CAR,NCR
province_list,{'ILOCOS NORTE': {'municipality_list': {'ADAMS...,{'BATANES': {'municipality_list': {'BASCO': {'...,{'AURORA': {'municipality_list': {'BALER': {'b...,{'BATANGAS': {'municipality_list': {'AGONCILLO...,{'MARINDUQUE': {'municipality_list': {'BOAC': ...,{'ALBAY': {'municipality_list': {'BACACAY': {'...,{'AKLAN': {'municipality_list': {'ALTAVAS': {'...,{'BOHOL': {'municipality_list': {'ALBURQUERQUE...,{'BILIRAN': {'municipality_list': {'ALMERIA': ...,{'ZAMBOANGA DEL NORTE': {'municipality_list': ...,{'BUKIDNON': {'municipality_list': {'BAUNGON':...,{'COMPOSTELA VALLEY': {'municipality_list': {'...,{'COTABATO (NORTH COT.)': {'municipality_list'...,{'AGUSAN DEL NORTE': {'municipality_list': {'B...,{'BASILAN': {'municipality_list': {'AKBAR': {'...,{'ABRA': {'municipality_list': {'BANGUED': {'b...,{'NATIONAL CAPITAL REGION - FOURTH DISTRICT': ...
